[Manual de uso do Pyomo](http://www.opl.ufc.br/files/Manual_Pyomo_2020.pdf)

[Vídeo aula - Modelando o problema do Transporte - parte 1](https://www.youtube.com/watch?v=McT1KDAE6xg&list=PLB0Bkje224EHp-IPd9i_FADBXXQcft5L6&index=14)

[Vídeo aula - Modelando o problema do Transporte - parte 2](https://www.youtube.com/watch?v=4t0OxOPkNVw&list=PLB0Bkje224EHp-IPd9i_FADBXXQcft5L6&index=16)

[Vídeo aula - Como modelar um problema lendo os dados de entrada a partir de arquivos csv](https://www.youtube.com/watch?v=peiG1_akAoU&list=PLB0Bkje224EHp-IPd9i_FADBXXQcft5L6&index=12)

[Vídeo aula - Lidando com vetores em Python](https://www.youtube.com/watch?v=DgQ7qlqkDQE&list=PLB0Bkje224EHp-IPd9i_FADBXXQcft5L6&index=9)

[Vídeo aula - Lidando com matrizes em Python](https://www.youtube.com/watch?v=XquctHGsEQk&list=PLB0Bkje224EHp-IPd9i_FADBXXQcft5L6&index=10)

[Livro - Pesquisa Operacional para Cursos de Engenharia](https://www.grupogen.com.br/pesquisa-operacional-para-cursos-de-engenharia)

[Contato](https://acsjunior.com)

(Belfiore e Fávero) A Karpet Ltda é uma empresa fabricante de autopeças, cujas sedes estão localizadas em Osasco, Sorocaba e São Sebastião. 

Seus clientes encontram-se em São Paulo, Rio de Janeiro e Curitiba. 

Os custos unitários de transporte de cada origem para cada destino, assim como a capacidade de cada fornecedor e a demanda de cada cliente, encontram-se na tabela abaixo. 

O objetivo é atender a demanda de cada consumidor final, respeitando as capacidades de fornecimento, de forma a minimizar o custo total de transporte. Modelar o problema de transporte.

| Origem | São Paulo | Rio de Janeiro | Curitiba |  Capacidade |
| --- | --- | --- | --- | --- | 
| Osasco | 12 | 22 | 30 | **100** |
| Sorocaba | 18 | 24 | 32 | **140** |
| São Sebastião | 22 | 15 | 34 | **160** |
| Demanda | **120** | **130** | **150** | 

### Índices / Conjuntos

$I \colon \text{Conjunto de fornecedores}, \{1, 2,\ldots, m\},$

$J \colon \text{Conjunto de consumidores}, \{1, 2, \ldots, n\}.$


### Parâmetros

$c_{ij} \colon \text{Custo unitário de transporte do fornecedor } i \in I \text{ para o consumidor } j \in J,$

$a_i \colon \text{Capacidade de abastecimento do fornecedor } i \in I,$

$b_j \colon \text{Demanda do consumidor } j \in J.$
 


### Variáveis de decisão

$x_{ij} \colon \text{Quantidades transportadas do fornecedor } i \in I \text{ para o consumidor }j \in J.$


### Formulação matemática

$\text{min }z = \sum\limits_{i \in I} \sum\limits_{j \in J} c_{ij} x_{ij}.$

sujeito a

$\sum\limits_{j \in J} x_{ij} \leq a_{i}, \forall \; i \in I,$

$\sum\limits_{i \in I} x_{ij} \geq b_{j}, \forall \; j \in J,$

$x_{ij} \geq 0 \;\; \forall i \in I \text{, }j \in J.$

In [1]:
import pyomo.environ as pyo

In [2]:
# Dados de entrada:

custos = [[12, 22, 30], 
          [18, 24, 32], 
          [22, 15, 34]]

capacidade = [100, 140, 160]

demanda = [120, 130, 150]

m = len(capacidade)
n = len(demanda)

In [3]:
# Declaração do modelo:

modelo = pyo.ConcreteModel()


# Índices:

modelo.I = pyo.RangeSet(m)
modelo.J = pyo.RangeSet(n)


# Parâmetros:

modelo.c = pyo.Param(modelo.I, modelo.J, initialize=lambda modelo, i, j: custos[i-1][j-1])
modelo.a = pyo.Param(modelo.I, initialize=lambda modelo, i: capacidade[i-1])
modelo.b = pyo.Param(modelo.J, initialize=lambda modelo, j: demanda[j-1])


# Variáveis de decisão:

modelo.x = pyo.Var(modelo.I, modelo.J, within=pyo.NonNegativeReals)


# Função objetivo:

def regra_z(mod):
    return pyo.summation(mod.c, mod.x)

# def regra_z(mod):
#     return sum(mod.c[i,j]*mod.x[i,j] for i in mod.I for j in mod.J)

modelo.z = pyo.Objective(rule=regra_z, sense=pyo.minimize) # minimize = default


# Restrições:

def regra_capacidade(mod, i):
    return sum(mod.x[i,j] for j in mod.J) <= mod.a[i]

modelo.restr_capacidade = pyo.Constraint(modelo.I, rule=regra_capacidade)


def regra_demanda(mod, j):
    return sum(mod.x[i,j] for i in mod.I) >= mod.b[j]

modelo.restr_demanda = pyo.Constraint(modelo.J, rule=regra_demanda)

In [4]:
# resultado = pyo.SolverFactory('glpk').solve(modelo)
resultado = pyo.SolverFactory('gurobi').solve(modelo)

In [5]:
modelo.x.pprint()

x : Size=9, Index=x_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 : 100.0 :  None : False : False : NonNegativeReals
    (1, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (1, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 1) :     0 :  20.0 :  None : False : False : NonNegativeReals
    (2, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 3) :     0 : 120.0 :  None : False : False : NonNegativeReals
    (3, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (3, 2) :     0 : 130.0 :  None : False : False : NonNegativeReals
    (3, 3) :     0 :  30.0 :  None : False : False : NonNegativeReals


In [6]:
modelo.z()

8370.0